# Annotation tool to create/correct 3D training labels

In [ ]:
import os
import numpy as np
import holoviews as hv
from improc.io import parse_collection, DCAccessor
DCAccessor.register()
from holoviews import opts

from inter_view.dashboards import OrthoAnnotationDashBoard

hv.extension('bokeh', logo=False, width=100)

# Annotations

Nuclei and lumen annotations are needed to train the segmentation models.

1. First place labels to be corrected in a correction folder, .e.g `nuclei_corrections`, `lumen_corrections`
1. Once annotated/corrected, move the files in the corresponding annot subfolder so that it can be used by the training task.

**Nuclei annotations**
- If no nuclei segmentation exist yet, use the partial annotations generated from the tracked seed as starting point (e.g. nuclei_weak_annot)
    - start the `NucleiSegmentationTrainingTask` to trigger the generation of the partial labels
- After a first prediction has been generated, use the segmentation viewer to pick and automatically copy files for correction


**Lumen annotation**
- If no lumen segmentation exist yet, set `n_blank_annot` in the config below to generate empty label from n randomly picked images
- After a first prediction has been generated, use the segmentation viewer to pick and automatically copy files for correction

# config

In [ ]:
# correction_layer = 'nuclei_corrections'
correction_layer = 'lumen_corrections'
n_blank_annot=0

nuclei_ch = 'Channel1-Deconv'
cell_ch = 'Channel0-Deconv'
spacing = (2,0.26,0.26)


basedir = '../data'
data_pattern = '{dataset_id}/{layer}/{fname_head}T{time:04d}.{ext}'
index = ['dataset_id', 'layer', 'time']


channel_config = {cell_ch:{'cmap':'red'},#, 'intensity_bounds':(100,60000), 'slider_limits':(0,6000)},
                  nuclei_ch:{'cmap':'gray'},#, 'intensity_bounds':(1400,20000), 'slider_limits':(0,6000)},
                  correction_layer:{'cmap':'glasbey_hv_16bit', 'raster_aggregator':'first', 'intensity_bounds':(-2**15,2**15-1), 'slider_limits':(-2**15,2**15-1), 'bitdepth':'int16', 'opacity':0.2}}

def set_inactive_tool(plot, element):
    plot.state.toolbar.active_inspect = None

opts.defaults(opts.Image('channel.{}'.format(correction_layer), clipping_colors={'min': (0, 0, 0, 0)}, clim=(0,2**16-1), tools=['hover'], hooks=[set_inactive_tool]))

opts.defaults(opts.Image('channel', frame_width=700))

# parse files

In [ ]:
def initialize_blank_annot(df, ref_layer, correction_layer, n_samples, seed=741):
    '''
    Note: this does not check if matching corrected annoations already exist.
    '''
    
    subdf = df.query('layer=="{}"'.format(ref_layer)).sample(n_samples, random_state=seed)
    for _,row in subdf.reset_index().iterrows():
        row_out = row.copy()
        row_out['layer'] = correction_layer
        img = row.dc.read()[0]
        row_out.dc.write(np.zeros(img.shape, dtype=np.int16)-1, compress=9, exist_ok=True)

In [ ]:
df = parse_collection(os.path.join(basedir, data_pattern), index)
df = df.dc[:,[nuclei_ch, cell_ch, correction_layer]]

if n_blank_annot > 0:
    initialize_blank_annot(df, nuclei_ch, correction_layer, n_samples=n_blank_annot)
    df = parse_collection(os.path.join(basedir, data_pattern), index)
    df = df.dc[:,[nuclei_ch, cell_ch, correction_layer]]

# filter samples needs to be corrected
df = df.groupby(['dataset_id','time'],).filter(lambda x: correction_layer in x.reset_index().layer.tolist())


df

# interactive dashboard
To start drawing select the 'Freehand Drawing Tool' in the toolbar on the right of the image.

- drawing label:
    - -1: un-annotated (does not contribute to the training loss)
    - 0: background
    - for nuclei:
        - [1...] instance labels. last one in the list is always unused
    - for lumen:
        - 1: lumen
        - 2: epithelium
- on click
    - \-: does nothing
    - pick label (pipette): sets label at the clicked position as drawing label
    - fill label (pot filling): fill the label at the clicked label with the current drawing labels (discontinuous)
- lock
    - background: prevent from drawing over the existing background
    - foreground: prevents from drawing over the existing labels
- draw in 3D: draw with thickness in the 3rd dimension. best used to draw an initial "ball" at the center of each nuclei
- save: saves the current annotation. The current annotation is also automatically saved when loading a new image
- discard changes: Cancels all changes made since the last save (i.e. not a granual ctrl+z!)

In [ ]:
db = OrthoAnnotationDashBoard(df=df,
                           multi_select_levels=['layer'],
                           channel_config=channel_config,
                           composite_channels=[nuclei_ch, cell_ch],
                           overlay_channels=[correction_layer],
                           spacing=spacing,
                           annot_channel=correction_layer)

db.panel().servable()